## Dependencies

In [1]:
# Confirmed needed dependencies
import zillow
import random
import pprint

# Seth 

# Research how to make this truly invisible on GitHub Repo
# from config import zwsid
# Paste in your zwsid in the meantime using the following line
# zwsid = ""

In [2]:
# Possibly needed, but not yet used dependencies
import pandas as pd
import requests
import json

In [ ]:
# Valerie

# Further setting up Notebook with Markdown sections, file outputs, action comments, etc.
# Create pandas dataframe 

## Geocoordinates of Austin

In [ ]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

# dependencies for geocoordinates generator
import random
import sys
import math

In [ ]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 10         #### Set radius in miles

points = 1000        #### Set number of lat,lon points to generate

output_file_name = "random_Lat_Lon.csv" #### Set output file name 

In [ ]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points = points+1


In [ ]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

In [ ]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(output_file_name, index=False, header=True)

## Plot coordinate points on map

Troy is working on this as a way to verify good random sampling of coordinate points and area selection, and as a way to get familar with plotting on maps


In [ ]:

import gmplot

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

#gmap.plot(latitudes, longitudes, 'cornflowerblue', edge_width=10)
gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"], '#3B0B39', size=10, marker=False)
#gmap.scatter(lat_lon_df["lat"], lat_lon_df["lon"] , 'k', marker=True)
#gmap.heatmap(, heat_lngs)

gmap.draw("mymap.html")

## Convert Coordinates to Residential Addresses

In [3]:
##### Yuta's Blocks #####

##### General Analysis on the Geopy Nominatim Method Below #####
# - You can filter the address by type='house', but some houses are labeled with type='yes'
# - 'Yes' type means means it's an unlabeled building.
# - Quite a bit of 'yes' type buildings actually hit a residential building, so we may get a high percentage of
#   residential buildings if we filter by 'house' or 'yes'.
# - Out of roughly 100 requests, around 70% will result in either 'house' or 'yes' type.
# - I can't seem to find the limit of API request, but we are throttled to 1 request per second.

# Testing Geopy Nominatim API Response
import geopy
from geopy.geocoders import Nominatim

geopy.geocoders.options.default_user_agent = "my-application"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

# Params 1 - Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== #01 - Home Response:")
response_1 = requests.get(url, params=params_1).json()
pp.pprint(response_1)
print("\n" + "="*60 + "\n")

===== #01 - Home Response:
{   'address': {   'country': 'USA',
                   'country_code': 'us',
                   'county': 'Williamson County',
                   'hamlet': 'Pond Springs',
                   'house_number': '12126',
                   'neighbourhood': "Hunter's Chase",
                   'postcode': '78729',
                   'road': 'Hunters Chase Drive',
                   'state': 'Texas',
                   'suburb': 'Jollyville'},
    'addresstype': 'place',
    'boundingbox': ['30.440962', '30.441162', '-97.777478', '-97.777278'],
    'category': 'place',
    'display_name': "12126, Hunters Chase Drive, Hunter's Chase, Jollyville, "
                    'Pond Springs, Williamson County, Texas, 78729, USA',
    'importance': '0',
    'lat': '30.441062',
    'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
               'https://osm.org/copyright',
    'lon': '-97.777378',
    'name': None,
    'osm_id': '3825454876',
    'osm_type': 'node',
 

In [4]:
# Import CSV, put into DataFrame
data_file = "random_Lat_Lon_sample.csv"
latlon_df = pd.read_csv(data_file)
latlon_df.head()

,lat,lon
0,30.291394,-97.852524
1,30.194187,-97.625975
2,30.240834,-97.777685
3,30.328267,-97.641748
4,30.239495,-97.862239


In [5]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

In [6]:
# Loop Request API and append to create new dataframe
query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
aType = []
lat = []
lon = []
neighborhood = []
addType = []

counter = 1
numRequests = len(lats)

print(f"Processing {numRequests} Requests...")

for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }
    
    response = requests.get(query_url, params=params).json()
    
    if response['type'] == 'house' or response['type'] == 'yes':
        postcode.append(response['address']['postcode'])
        aType.append(response['type'])
        lat.append(response['lat'])
        lon.append(response['lon'])
    
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
        try:
            addType.append(response['addresstype'])
        except (KeyError, IndexError):
            addType.append("NA")
    
        print(f"Processed Record {counter} of {numRequests}.")
        counter = counter + 1
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
        counter = counter + 1

Processing 99 Requests...
Processed Record 1 of 99.
Wrong Type - Skipped Record 2 of 99.
Wrong Type - Skipped Record 3 of 99.
Processed Record 4 of 99.
Processed Record 5 of 99.
Wrong Type - Skipped Record 6 of 99.
Processed Record 7 of 99.
Processed Record 8 of 99.
Processed Record 9 of 99.
Processed Record 10 of 99.
Wrong Type - Skipped Record 11 of 99.
Processed Record 12 of 99.
Processed Record 13 of 99.
Processed Record 14 of 99.
Processed Record 15 of 99.
Processed Record 16 of 99.
Processed Record 17 of 99.
Processed Record 18 of 99.
Processed Record 19 of 99.
Processed Record 20 of 99.
Wrong Type - Skipped Record 21 of 99.
Processed Record 22 of 99.
Processed Record 23 of 99.
Wrong Type - Skipped Record 24 of 99.
Wrong Type - Skipped Record 25 of 99.
Processed Record 26 of 99.
Processed Record 27 of 99.
Wrong Type - Skipped Record 28 of 99.
Wrong Type - Skipped Record 29 of 99.
Processed Record 30 of 99.
Wrong Type - Skipped Record 31 of 99.
Processed Record 32 of 99.
Wrong Typ

In [7]:
# Create a dataframe with all addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
    "type": aType,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
    "address type": addType
})
address_df

,house #,street,zipcode,type,lat,lon,neighborhood,address type
0,7904,Barton Club Drive,78735,house,30.291968,-97.852151,NA,place
1,9345,US Highway 290,78754,yes,30.328789,-97.6420199618697,Chimney Hills,building
2,6550,West William Cannon Drive,78735,house,30.2393099539122,-97.8619814107644,Legend Oaks,place
3,5104,Raton Pass,78724,yes,30.312713,-97.6590784469697,Northridge Park,building
4,4017,Greystone Drive,78731,yes,30.3633565,-97.7627070161153,Mesa Forest,building
5,8828,Research Boulevard,78757,yes,30.3675846,-97.7194690492224,Wooten,building
6,7803,FM 1625,78744,house,30.1365838876672,-97.7052619134836,NA,place
7,7201,Rain Creek Parkway,78759,yes,30.415588,-97.7732451736842,NA,building
8,3201,Duval Road,78759-3504,yes,30.4092425,-97.7187060969141,North Star,building
9,1417,Berkshire Drive,78723,yes,30.316552,-97.6961592194393,Windsor Park,building


In [8]:
# write a CSV file of addresses
address_df.to_csv("address_list_sample.csv", index=False, header=True)

In [10]:
# Pull from CSV
address_csv = "address_list_sample.csv"
address_sample = pd.read_csv(address_csv)
address_sample.head()

,house #,street,zipcode,type,lat,lon,neighborhood,address type
0,7904,Barton Club Drive,78735,house,30.291968,-97.852151,NaN,place
1,9345,US Highway 290,78754,yes,30.328789,-97.642020,Chimney Hills,building
2,6550,West William Cannon Drive,78735,house,30.239310,-97.861981,Legend Oaks,place
3,5104,Raton Pass,78724,yes,30.312713,-97.659078,Northridge Park,building
4,4017,Greystone Drive,78731,yes,30.363357,-97.762707,Mesa Forest,building


In [16]:
# Map out the sample with gmplot
import gmplot

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(address_sample["lat"], address_sample["lon"], 'red', size=80, marker=False)

gmap.draw("myaddressmap.html")

In [ ]:
# Kat's block

## Zillow API Calls using Address and Zipcode

In [ ]:
address = "929 Cavalier Ln" 
zipcode = "78757"

pp = pprint.PrettyPrinter(indent=4)

api = zillow.ValuationApi()


#This appears to be the ideal search to use with the most flushed out info
deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
pp.pprint(deep_results.get_dict())


# This appears to return the same as .GetZEstimate(),
# but has less accurate "extended_data" than .GetDeepSearchResults
# data = api.GetSearchResults(zwsid, address, zipcode)
# pp.pprint(data.get_dict())


# This appears to return the same as .GetSearchResults()
# detail_data = api.GetZEstimate(zwsid, data.zpid)
# pp.pprint(detail_data.get_dict())

## Calculate Value per Sqft

In [ ]:
# Seth

#Create function to calculate value per sqft using "zestimate"["amount"] / "extended_data"["finished_sqft"]
